<a href="https://colab.research.google.com/github/jay-kanakia/GenAI/blob/main/03_Pytorch_training_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [5]:
df.sample(3)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
554,B,12.88,28.92,82.50,514.3,0.08123,0.05824,0.06195,0.02343,0.1566,...,13.89,35.74,88.84,595.7,0.1227,0.1620,0.2439,0.06493,0.2372,0.07242
98,B,11.60,12.84,74.34,412.6,0.08983,0.07525,0.04196,0.03350,0.1620,...,13.06,17.16,82.96,512.5,0.1431,0.1851,0.1922,0.08449,0.2772,0.08756
452,B,12.00,28.23,76.77,442.5,0.08437,0.06450,0.04055,0.01945,0.1615,...,13.09,37.88,85.07,523.7,0.1208,0.1856,0.1811,0.07116,0.2447,0.08194


In [29]:
df.shape

(569, 31)

In [30]:
df.isnull().sum()

,0
diagnosis,0
radius_mean,0
texture_mean,0
perimeter_mean,0
area_mean,0
smoothness_mean,0
compactness_mean,0
concavity_mean,0
concave points_mean,0
symmetry_mean,0


In [14]:
X=df.iloc[:,1:]
y=df.iloc[:,0]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

ss=StandardScaler()
X_train_scaled=ss.fit_transform(X_train)
X_test_scaled=ss.transform(X_test)

le=LabelEncoder()
y_train_transformed=le.fit_transform(y_train)
y_test_transformed=le.transform(y_test)

In [40]:
X_train_tensor=torch.from_numpy(X_train_scaled).float()
X_test_tensor=torch.from_numpy(X_test_scaled).float()
y_train_tensor=torch.from_numpy(y_train_transformed).float()
y_test_tensor=torch.from_numpy(y_test_transformed).float()

In [61]:
X_train_scaled.dtype

dtype('float64')

In [62]:
X_train_tensor.dtype

torch.float32

In [42]:
X_train_tensor.shape

torch.Size([455, 30])

In [47]:
class MySimpleNN():

  def __init__(self,X):
    self.weights=torch.rand(X.shape[1],1,dtype=torch.float32,requires_grad=True)
    self.bias=torch.zeros(1,dtype=torch.float32,requires_grad=True)

  def forward(self,X):
    z=torch.matmul(X,self.weights)+self.bias
    y_pred=torch.sigmoid(z)
    return y_pred

  def loss_function(self, y_pred, y):
    # Clamp predictions to avoid log(0)
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

    # Calculate loss
    loss = -(y * torch.log(y_pred) + (1 - y) * torch.log(1 - y_pred)).mean()
    return loss

In [63]:
learning_rate=0.01
epochs=2000

model=MySimpleNN(X_train_tensor)

for epoch in range(epochs):

  y_pred=model.forward(X_train_tensor)

  loss=model.loss_function(y_pred,y_train_tensor)

  loss.backward()

  with torch.no_grad():
    model.weights -= learning_rate * model.weights.grad
    model.bias    -= learning_rate * model.bias.grad

    # Zero gradients after update
    model.weights.grad.zero_()
    model.bias.grad.zero_()

  print(f"for epoch {epoch+1} loss is {loss.item()}")

for epoch 1 loss is 3.104005813598633
for epoch 2 loss is 3.088681221008301
for epoch 3 loss is 3.073618173599243
for epoch 4 loss is 3.0583548545837402
for epoch 5 loss is 3.04158616065979
for epoch 6 loss is 3.0269503593444824
for epoch 7 loss is 3.010054111480713
for epoch 8 loss is 2.9946956634521484
for epoch 9 loss is 2.9800565242767334
for epoch 10 loss is 2.9634108543395996
for epoch 11 loss is 2.9487664699554443
for epoch 12 loss is 2.9323959350585938
for epoch 13 loss is 2.916835308074951
for epoch 14 loss is 2.900510311126709
for epoch 15 loss is 2.8858094215393066
for epoch 16 loss is 2.869412422180176
for epoch 17 loss is 2.853649377822876
for epoch 18 loss is 2.8378214836120605
for epoch 19 loss is 2.8220064640045166
for epoch 20 loss is 2.8051443099975586
for epoch 21 loss is 2.7891716957092285
for epoch 22 loss is 2.772996425628662
for epoch 23 loss is 2.756586790084839
for epoch 24 loss is 2.7392709255218506
for epoch 25 loss is 2.7230310440063477
for epoch 26 loss is 

In [64]:
print(model.weights.requires_grad, model.bias.requires_grad)


True True


In [65]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')


Accuracy: 0.6228070259094238
